In [2]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import os
from datetime import datetime, date, timedelta
from typing import Any, List, Dict, Union, Tuple
import pandas as pd

## Enable full-width pandas dataframe columns so the paths and filenames always fit
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)
# /umms-kdiba/Data/Output/gen_scripts/

import globus_sdk
from globus_sdk import AccessTokenAuthorizer, TransferClient, TransferData
from globus_sdk.scopes import TransferScopes
from attrs import define, field, Factory

from phoglobushelpers.PhoGlobusHelper import GlobusConnector, KnownEndpoints
from phoglobushelpers.compatibility_objects.Bookmarks import Bookmark, BookmarkList
from phoglobushelpers.compatibility_objects.Files import File, FilesystemDataType, FileList
from phoglobushelpers.compatibility_objects.Tasks import FatalError, Task, TaskList

Automatic pdb calling has been turned OFF


In [3]:
connect_man = GlobusConnector.login_and_get_transfer_client()
transfer_client = connect_man.transfer_client
connect_man.list_endpoints()


Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=769d24e1-d1cc-4198-9ff7-2626485da449&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=sQa9kbwYbTYzoqlujYZVvM3UYYBqjW_1SXt0Ocgyf-E&code_challenge_method=S256&access_type=offline
	 Copied url to clipboard!


My Endpoints:
[c3a90494-2555-11ec-a47d-a50ad076c282] aleinbook-linux
[df4149a0-9925-11ee-be43-c52a29481bea] Aleinware
[84991054-07b4-11ed-8d83-a54cf61939f8] Apogee
[6d0251c4-2585-11ec-9e35-3df4ed83d858] Cubix Win10
[c3578f36-42a8-11ec-a50f-b537d6c07c1d] FlatEdge_Server
[af3fcfce-f664-11ed-9a7d-83ef71fbf0ae] LNX00052_Fedora
[44727efc-c21b-11ee-89e3-5130436d8581] LNX00052_Ubuntu
[2621bd72-f688-11ee-b6ec-473d136f742f] MM2
[20c84240-1eb1-11eb-81b7-0e2f230cc907] Pho Personal Laptop
[560d3a12-be20-11ed-9916-cb2cff506ca5] Pho_Personal_Testix
[cee0aaec-3e3b-11ef-9637-453c3ae125a5] RDLU0039_DibaLabWorkstation
[debb635c-2556-11ec-a47d-a50ad076c282] rMBP Pink Dot
[e414f584-2556-11ec-a0a7-6b21ca6daf73] rMBP Pink Dot
[41b8fb12-516b-11ee-8a4b-b1038a0991ac] rMBP-PinkDot
[3fc00ebe-f142-11ee-91fe-472b0fe4395a] TempVM_VS_NonGPU


# Tasks

In [ ]:
task_id = "a5bba88c-fba0-11ee-9063-1f4fee4027cc"

In [44]:
tasks_list = connect_man.get_tasks()
tasks_list_df = tasks_list.to_dataframe()
tasks_list_df

,bytes_checksummed,bytes_transferred,canceled_by_admin,canceled_by_admin_message,command,completion_time,deadline,delete_destination_extra,destination_endpoint,destination_endpoint_display_name,...,subtasks_retrying,subtasks_skipped_errors,subtasks_succeeded,subtasks_total,symlinks,sync_level,task_id,type,username,verify_checksum
0,0,901832024,None,None,API 0.10 go,2024-04-11T14:17:42+00:00,2024-07-08T14:19:23+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#3fc00ebe-f142-11ee-91fe-472b0fe4395a,TempVM_VS_NonGPU,...,0,2537,11077,56093,0,NaN,cca332b4-f1ee-11ee-b6d4-473d136f742f,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,False
1,0,128743332848,None,None,API 0.10 go,2024-04-11T15:41:54+00:00,2024-04-14T15:10:30+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#20c84240-1eb1-11eb-81b7-0e2f230cc907,Pho Personal Laptop,...,0,0,10846,11156,0,2.0,fb4504d4-f80c-11ee-b6f7-473d136f742f,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
2,0,88033380004,None,None,API 0.10 go,2024-04-11T17:52:22+00:00,2024-04-14T17:06:17+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#20c84240-1eb1-11eb-81b7-0e2f230cc907,Pho Personal Laptop,...,0,0,37,205,0,2.0,e7dd3d50-f819-11ee-921d-472b0fe4395a,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
3,0,7052098033,None,None,API 0.10 go,2024-04-12T12:31:21+00:00,2024-04-13T12:23:59+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#84991054-07b4-11ed-8d83-a54cf61939f8,Apogee,...,0,0,1465,1465,0,NaN,8951e322-f8c7-11ee-9220-472b0fe4395a,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
4,0,10598024699,None,None,API 0.10 go,2024-04-12T21:09:42+00:00,2024-04-13T20:45:29+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#84991054-07b4-11ed-8d83-a54cf61939f8,Apogee,...,0,0,102,162,0,2.0,9756ad39-f90d-11ee-b6fb-473d136f742f,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
5,0,1897803436,None,None,API 0.10 go,2024-04-12T21:10:42+00:00,2024-04-13T21:09:58+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#84991054-07b4-11ed-8d83-a54cf61939f8,Apogee,...,0,0,162,162,0,2.0,03cc7dd0-f911-11ee-b6fb-473d136f742f,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
6,0,6696588836,None,None,API 0.10 go,2024-04-12T22:34:37+00:00,2024-04-15T22:07:15+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#84991054-07b4-11ed-8d83-a54cf61939f8,Apogee,...,0,0,58,58,0,2.0,5e67f808-f918-11ee-9222-472b0fe4395a,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
7,0,9869341597,None,None,API 0.10 go,2024-04-15T12:07:12+00:00,2024-04-16T12:01:42+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#84991054-07b4-11ed-8d83-a54cf61939f8,Apogee,...,0,0,3100,3100,0,2.0,ebb4e4e2-fb1f-11ee-9063-1f4fee4027cc,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
8,0,3379152716,None,None,API 0.10 go,2024-04-15T20:28:04+00:00,2024-04-16T20:26:36+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#84991054-07b4-11ed-8d83-a54cf61939f8,Apogee,...,0,0,402,402,0,2.0,742635ba-fb66-11ee-9063-1f4fee4027cc,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
9,0,719838100,None,None,API 0.10 go,2024-04-15T21:58:48+00:00,2024-04-16T21:58:23+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#84991054-07b4-11ed-8d83-a54cf61939f8,Apogee,...,0,0,196,196,0,2.0,46eb8d54-fb73-11ee-922e-472b0fe4395a,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True


In [ ]:
tasks_list_df

In [45]:
# [Bookmark(bookmark_id='9a8eccc8-2558-11ec-a0a7-6b21ca6daf73', name='aleinbook-linux - PhoPy3DPositionAnalysis', endpoint_id='c3a90494-2555-11ec-a47d-a50ad076c282', path='/~/repo/PhoPy3DPositionAnalysis/'),
#  Bookmark(bookmark_id='3453290e-9926-11ee-8c90-fd88ce9321ad', name='Aleinware SmallSSD', endpoint_id='df4149a0-9925-11ee-be43-c52a29481bea', path='/F/'),
#  Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/'),
#  Bookmark(bookmark_id='fa915572-43a0-11ee-a06e-eb83daae1adf', name='Apogee DibaLabGDriveMount', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/L/'),
#  Bookmark(bookmark_id='4f1229fe-3eba-11ef-9637-453c3ae125a5', name='Apogee - gen_scripts', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/K/scratch/gen_scripts/'),
#  Bookmark(bookmark_id='965ffa0e-f90d-11ee-9222-472b0fe4395a', name='Apogee - K - collected_outputs', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/K/scratch/collected_outputs/'),
#  Bookmark(bookmark_id='a267ed38-8d7f-11ee-be03-c52a29481bea', name='Apogee - K - UE5_3 - Plugins', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/K/FastSwap/ProgramFiles/UE_5.3/Engine/Plugins/'),
#  Bookmark(bookmark_id='63655524-164e-11ee-bfba-195c41bc0be4', name='Apogee - PersonalFastSwapShare', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/FastSwap/PersonalFastSwapShare/'),
#  Bookmark(bookmark_id='1c725da6-203a-11ee-80bc-a3018385fcef', name='Apogee - ProgrammaticDisplayFunctionTesting', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/C/Users/pho/repos/Spike3DWorkEnv/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='eeab2364-f8c2-11ee-921f-472b0fe4395a', name='Apogee - Spike3DWorkEnv - Output', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/C/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/'),
#  Bookmark(bookmark_id='66ead9ec-2e56-11ee-b3ed-ad2493b708f4', name='Apogee - Unreal Root', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/E/UNREAL/'),
#  Bookmark(bookmark_id='78ca952e-2586-11ec-9e35-3df4ed83d858', name='Cubix Win10 PhoPy3DPositionAnalysis2021', endpoint_id='6d0251c4-2585-11ec-9e35-3df4ed83d858', path='/~/repos/PhoPy3DPositionAnalysis2021/'),
#  Bookmark(bookmark_id='06f3fc6e-b65d-11ee-84a8-f1441124ff66', name='Day5TwoNovel', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/Bapun/RatS/Day5TwoNovel/'),
#  Bookmark(bookmark_id='c23b0afa-3261-11ef-b82c-1dd816fe311b', name='DellaKouroshReMazeDataShare', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/_TEMP/DellaKouroshReMazeDataShare/'),
#  Bookmark(bookmark_id='99efa634-3ead-11ef-888b-2b3122c1d121', name='Greatlakes gen_scripts', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/Output/gen_scripts/'),
#  Bookmark(bookmark_id='431c951e-96c6-11ed-9b93-19370d280681', name='kdiba DataDen Data', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/'),
#  Bookmark(bookmark_id='8ce139f8-9d46-11ed-a2a2-8383522b48d9', name='KDIBA Lab Turbo', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/'),
#  Bookmark(bookmark_id='91181e28-f90d-11ee-9222-472b0fe4395a', name='KDIBA Lab Turbo - collected_outputs', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/Output/collected_outputs/'),
#  Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/'),
#  Bookmark(bookmark_id='fa5567f4-5d97-11ee-8e55-e7956a689924', name='LNX00052_Fedora - BETAMAX 20TB Disk', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/mnt/BETAMAX/'),
#  Bookmark(bookmark_id='de42717e-5d97-11ee-8e55-e7956a689924', name='LNX00052_Fedora - DELTAMAX 20TB Disk', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/mnt/DELTAMAX/'),
#  Bookmark(bookmark_id='7b24feb0-5e26-11ee-8776-1dc3121de006', name='LNX00052_Fedora - FastData Global Data Root', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/home/halechr/FastData/'),
#  Bookmark(bookmark_id='e50e6dd2-5d97-11ee-8e55-e7956a689924', name='LNX00052_Fedora - GAMMAMAX 20TB Disk', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/mnt/GAMMAMAX/'),
#  Bookmark(bookmark_id='6e727c22-5d97-11ee-8e55-e7956a689924', name='LNX00052_Fedora - GDrive_Diba_Shared Data', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/media/MAX/cloud/GDrive_Diba_Shared/Data/'),
#  Bookmark(bookmark_id='211f7d4e-478a-11ee-a072-eb83daae1adf', name='LNX00052_Fedora - HugePort 20TB Disk', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/media/HugePort/'),
#  Bookmark(bookmark_id='82d87ba0-5d59-11ee-8e55-e7956a689924', name='LNX00052_Fedora - HUUUGE 20TB Disk', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/mnt/wwn-0x5000cca2b3c3f6f7-part1/'),
#  Bookmark(bookmark_id='248a9a30-164e-11ee-bfba-195c41bc0be4', name='LNX00052_Fedora - MAX', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/media/MAX/'),
#  Bookmark(bookmark_id='fb16ea72-8d6d-11ee-8c73-fd88ce9321ad', name='LNX00052_Fedora - MAX - UnrealProjects', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/media/MAX/Personal/UnrealProjects/'),
#  Bookmark(bookmark_id='c817bbf0-4da9-11ee-a088-eb83daae1adf', name='LNX00052_Fedora - New ProgrammaticDisplayFunctionTesting', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='af68b1ea-20c7-11ee-818f-195c41bc0be4', name='LNX00052_Fedora - ProgrammaticDisplayFunctionTesting', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/home/halechr/repo/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='b214aca0-4da9-11ee-a088-eb83daae1adf', name='LNX00052_Fedora - Spike3D Repo', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/home/halechr/repos/Spike3D/'),
#  Bookmark(bookmark_id='ca95eb76-c21c-11ee-84b4-f1441124ff66', name='LNX00052_Ubuntu', endpoint_id='44727efc-c21b-11ee-89e3-5130436d8581', path='/~/'),
#  Bookmark(bookmark_id='25a26f84-cd13-11ee-a48f-73d2c6fd707d', name='LNX00052_Ubuntu - UnrealProjects', endpoint_id='44727efc-c21b-11ee-89e3-5130436d8581', path='/media/halechr/MAX/Personal/UnrealProjects/'),
#  Bookmark(bookmark_id='b6d70674-3144-11ee-87a9-4dfadf03ac7e', name='lsa-halechr Personal Turbo', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/lsa-halechr/'),
#  Bookmark(bookmark_id='034e1614-3c19-11ec-90c8-7fa38c35632a', name='NYU Langone Health DTN1 - Buzsakilab Public Datasets', endpoint_id='188a6110-96db-11eb-b7a9-f57b2d55370d', path='/'),
#  Bookmark(bookmark_id='bbb4a1a8-f0b9-11ee-91fe-472b0fe4395a', name='Pho Personal Laptop - DibaMac', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Volumes/DibaMac/'),
#  Bookmark(bookmark_id='b0066292-f0b9-11ee-91fe-472b0fe4395a', name='Pho Personal Laptop - halechrGDrive', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Volumes/DibaMac/Backups/Pho/'),
#  Bookmark(bookmark_id='1805df24-e618-11ed-9a63-83ef71fbf0ae', name='Pho Personal Laptop - PegasusR6 - Data', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Volumes/Pegasus R6/Data/'),
#  Bookmark(bookmark_id='3a94f9b2-03ef-11ef-82a6-57b5c7e2fa2b', name='Pho Personal Laptop - SwapSSD Data', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Volumes/SwapSSD/Data/'),
#  Bookmark(bookmark_id='be76c06a-51e2-11ee-a0a8-eb83daae1adf', name='rMBP-PinkDot - LargeDisk', endpoint_id='41b8fb12-516b-11ee-8a4b-b1038a0991ac', path='/Volumes/LargeDisk/'),
#  Bookmark(bookmark_id='883e37d8-f142-11ee-904b-1f4fee4027cc', name='TempVM_VS_NonGPU', endpoint_id='3fc00ebe-f142-11ee-91fe-472b0fe4395a', path='/'),
#  Bookmark(bookmark_id='d85320ee-f142-11ee-904b-1f4fee4027cc', name='TempVM_VS_NonGPU - GDrive - WatsonLab2019-2020', endpoint_id='3fc00ebe-f142-11ee-91fe-472b0fe4395a', path='/L/MED-WatsonDigitalPhenotyping/FromGDrive2024/Watson Lab 2019-2020/'),
#  Bookmark(bookmark_id='90d94d74-f142-11ee-904b-1f4fee4027cc', name='TempVM_VS_NonGPU - K - GDrive', endpoint_id='3fc00ebe-f142-11ee-91fe-472b0fe4395a', path='/K/'),
#  Bookmark(bookmark_id='c1dae626-f142-11ee-904b-1f4fee4027cc', name='TempVM_VS_NonGPU - L - Dropbox', endpoint_id='3fc00ebe-f142-11ee-91fe-472b0fe4395a', path='/L/'),
#  Bookmark(bookmark_id='99ce1e08-bdf1-11ed-8cec-f9fa098153fc', name='UMich Diba Turbo - Pho Directory', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Pho/'),
#  Bookmark(bookmark_id='b1668f08-2036-11ee-80bc-a3018385fcef', name='umich#greatlakes - halechr homedir', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/home/halechr/'),
#  Bookmark(bookmark_id='f479daf4-2039-11ee-80bc-a3018385fcef', name='umich#greatlakes - ProgrammaticDisplayFunctionTesting', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='d7c1b398-4ca6-11ee-a085-eb83daae1adf', name='umich#greatlakes - Scratch', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/scratch/kdiba_root/'),
#  Bookmark(bookmark_id='bef81650-96c5-11ed-9b93-19370d280681', name='umms-dibalab DataDen', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/')]

bookmark_list = connect_man.get_bookmarks()
bookmark_list.DATA

[Bookmark(bookmark_id='9a8eccc8-2558-11ec-a0a7-6b21ca6daf73', name='aleinbook-linux - PhoPy3DPositionAnalysis', endpoint_id='c3a90494-2555-11ec-a47d-a50ad076c282', path='/~/repo/PhoPy3DPositionAnalysis/'),
 Bookmark(bookmark_id='3453290e-9926-11ee-8c90-fd88ce9321ad', name='Aleinware SmallSSD', endpoint_id='df4149a0-9925-11ee-be43-c52a29481bea', path='/F/'),
 Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/'),
 Bookmark(bookmark_id='fa915572-43a0-11ee-a06e-eb83daae1adf', name='Apogee DibaLabGDriveMount', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/L/'),
 Bookmark(bookmark_id='4f1229fe-3eba-11ef-9637-453c3ae125a5', name='Apogee - gen_scripts', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/K/scratch/gen_scripts/'),
 Bookmark(bookmark_id='965ffa0e-f90d-11ee-9222-472b0fe4395a', name='Apogee - K - collected_outputs', endpoint_id='84991054-07b4-11ed-8d83-a54cf61

In [ ]:
lab_Turbo_data_folder_bookmark = Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')
lab_DataDen_data_folder_bookmark = Bookmark(bookmark_id='431c951e-96c6-11ed-9b93-19370d280681', name='kdiba DataDen Data', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/')

# Figure Outputs:
lab_Greatlakes_ProgrammaticDisplayFunctionTesting_bookmark = Bookmark(bookmark_id='f479daf4-2039-11ee-80bc-a3018385fcef', name='umich#greatlakes - ProgrammaticDisplayFunctionTesting', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/')
lab_LNX00052_Fedora_ProgrammaticDisplayFunctionTesting_bookmark = Bookmark(bookmark_id='af68b1ea-20c7-11ee-818f-195c41bc0be4', name='LNX00052_Fedora - ProgrammaticDisplayFunctionTesting', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/home/halechr/repo/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/')

## Apogee:
home_Apogee_ProgrammaticDisplayFunctionTesting_bookmark = Bookmark(bookmark_id='1c725da6-203a-11ee-80bc-a3018385fcef', name='Apogee - ProgrammaticDisplayFunctionTesting', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/C/Users/pho/repos/Spike3DWorkEnv/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/')
apogee_collected_outputs_folder_bookmark = Bookmark(bookmark_id='965ffa0e-f90d-11ee-9222-472b0fe4395a', name='Apogee - K - collected_outputs', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/K/scratch/collected_outputs/')

## `gen_scripts` folders:
apogee_gen_scripts_folder_bookmark = Bookmark(bookmark_id='4f1229fe-3eba-11ef-9637-453c3ae125a5', name='Apogee - gen_scripts', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/K/scratch/gen_scripts/')
lab_Greatlakes_gen_scripts = Bookmark(bookmark_id='99efa634-3ead-11ef-888b-2b3122c1d121', name='Greatlakes gen_scripts', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/Output/gen_scripts/')
# lab_Greatlakes_gen_scripts, apogee_gen_scripts_folder_bookmark

## OUTPUTS: lab_Turbo_data_folder_bookmark, lab_DataDen_data_folder_bookmark, lab_Greatlakes_gen_scripts, apogee_gen_scripts_folder_bookmark, apogee_collected_outputs_folder_bookmark

Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')

In [26]:
# target_bookmark: Bookmark = lab_DataDen_data_folder_bookmark
# target_bookmark: Bookmark = lab_Turbo_data_folder_bookmark
target_bookmark = lab_Greatlakes_ProgrammaticDisplayFunctionTesting_bookmark
# target_bookmark = lab_Greatlakes_gen_scripts
target_bookmark

Bookmark(bookmark_id='f479daf4-2039-11ee-80bc-a3018385fcef', name='umich#greatlakes - ProgrammaticDisplayFunctionTesting', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/')

In [27]:
file_list = connect_man.list_files(endpoint=target_bookmark.endpoint_id, path=target_bookmark.path, start_date=None, end_date=None, should_list_recursively=False)
file_list.to_dataframe()

,type,parent_path,name,last_modified,size


In [ ]:
recent_file_list = connect_man.list_files(endpoint=target_bookmark.endpoint_id, path=target_bookmark.path, start_date="2023-08-07", end_date=None, should_list_recursively=True)
recent_file_list.to_dataframe() #.columns

In [ ]:
recent_file_list = connect_man.list_files(endpoint=home_Apogee_ProgrammaticDisplayFunctionTesting_bookmark.endpoint_id, path=home_Apogee_ProgrammaticDisplayFunctionTesting_bookmark.path, start_date="2023-08-07", end_date=None, should_list_recursively=True)
recent_file_list.to_dataframe() #.columns

In [ ]:
recent_file_list = connect_man.list_files(endpoint=target_bookmark.endpoint_id, path=target_bookmark.path, start_date="2023-08-07", end_date=None, should_list_recursively=True)
recent_file_list # 1m 44.7 sec

In [ ]:
recent_file_list_df = recent_file_list.to_dataframe()
recent_file_list_df

In [ ]:
recent_file_list.path

# Loading filelists:

In [ ]:
from phoglobushelpers.path_helpers import convert_filelist_to_new_parent, read_lines_from_file, find_matching_parent_path, known_global_data_root_parent_paths


filelist_source_path = Path('../filelists/fileList_Greatlakes_2023-08-07.txt').resolve()
src_lines = read_lines_from_file(filelist_source_path)

filelist_dest_path = '../filelists/dest_fileList_Apogee_2023-08-07.txt'
dest_lines = read_lines_from_file(filelist_dest_path)


filelist_source_path = Path('../filelists/fileList_Greatlakes_HDF5_2023-08-07.txt').resolve()
src_lines = read_lines_from_file(filelist_source_path)


In [ ]:
filelist_source_path = Path('../filelists/fileList_Greatlakes_pkls_2023-08-07.txt').resolve()
src_lines = read_lines_from_file(filelist_source_path)
src_lines

In [ ]:
filelist_source_path = Path('../filelists/fileList_Greatlakes_HDF5_2023-08-07.txt').resolve()
src_lines = read_lines_from_file(filelist_source_path)
src_lines

In [ ]:
src_lines = [a_line.replace('/nfs/turbo/umms-kdiba/Data/', '/umms-kdiba/Data/', 1) for a_line in src_lines]
src_lines

dest_lines = [a_line.replace('/umms-kdiba/Data/', '/~/W/Data/', 1) for a_line in src_lines]
dest_lines

In [ ]:
from pathlib import Path


curr_filelist = [Path(a_line) for a_line in src_lines]

desired_global_data_root_parent_path = Path('/~/W/Data/')
prev_global_data_root_parent_path = find_matching_parent_path(known_global_data_root_parent_paths, curr_filelist[0]) # TODO: assumes all have the same root, which is a valid assumption so far. ## prev_global_data_root_parent_path should contain the matching path from the list.
assert prev_global_data_root_parent_path is not None, f"No matching root parent path could be found!!"
converted_dest_paths = convert_filelist_to_new_parent(curr_filelist, original_parent_path=prev_global_data_root_parent_path, dest_parent_path=desired_global_data_root_parent_path)
dest_lines = [str(a_path) for a_path in converted_dest_paths]
dest_lines
converted_dest_paths[0]

In [ ]:
from phoglobushelpers.path_helpers import convert_filelist_to_new_parent

# source_parent_path = Path(r'/media/MAX/cloud/turbo/Data')
source_parent_path = Path(r'/nfs/turbo/umms-kdiba/Data')
dest_parent_path = Path(r'/~/W/Data/')
# # Build the destination filelist from the source_filelist and the two paths:
filelist_source = hdf5_output_paths
filelist_dest_paths = convert_filelist_to_new_parent(filelist_source, original_parent_path=source_parent_path, dest_parent_path=dest_parent_path)
filelist_dest_paths

In [ ]:
apogee_data_folder_bookmark = Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/')
lab_Turbo_data_folder_bookmark = Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')

# endpoint_relative_src_lines = [a_line.replace('/umms-kdiba/Data/', '/', 1) for a_line in src_lines]
# endpoint_relative_dest_lines = [a_line.replace('/~/W/Data/', '/', 1) for a_line in dest_lines]

endpoint_relative_src_lines = [a_line for a_line in src_lines]
endpoint_relative_dest_lines = [a_line for a_line in dest_lines]
_transfer_task = connect_man.batch_transfer_files(source_endpoint=lab_Turbo_data_folder_bookmark.endpoint_id, destination_endpoint=apogee_data_folder_bookmark.endpoint_id, filelist_source=endpoint_relative_src_lines, filelist_dest=endpoint_relative_dest_lines, max_single_file_wait_time_seconds=60.0)
_transfer_task

# 2024-07-10 - Pulling Greatlakes Batch Run Logs

In [ ]:
from phoglobushelpers.PhoGlobusHelper import get_greatlakes_gen_scripts_log_files

lab_Greatlakes_gen_scripts = Bookmark(bookmark_id='99efa634-3ead-11ef-888b-2b3122c1d121', name='Greatlakes gen_scripts', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/Output/gen_scripts/')
all_log_file_df, most_recent_only_log_file_df = get_greatlakes_gen_scripts_log_files(connect_man)
most_recent_only_log_file_df

## Copy these most recent logs to Apogee

In [67]:
## INPUTS: most_recent_only_log_file_df
apogee_gen_scripts_folder_bookmark = Bookmark(bookmark_id='4f1229fe-3eba-11ef-9637-453c3ae125a5', name='Apogee - gen_scripts', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/K/scratch/gen_scripts/')

most_recent_only_log_file_paths: List[Path] = [Path(row.parent_path).joinpath(row.name) for row in most_recent_only_log_file_df.itertuples(name='LogFile')]

## INPUTS: most_recent_only_log_file_paths

src_lines: List[str] = [str(a_line.as_posix()) for a_line in most_recent_only_log_file_paths]
dest_lines: List[str] = [a_line.replace('/umms-kdiba/Data/Output/gen_scripts/', '/K/scratch/gen_scripts/', 1) for a_line in src_lines]

## INPUTS: src_lines, dest_lines, apogee_gen_scripts_folder_bookmark, lab_Greatlakes_gen_scripts
endpoint_relative_src_lines = [a_line for a_line in src_lines]
endpoint_relative_dest_lines = [a_line for a_line in dest_lines]

# ## Async:
# _transfer_task = connect_man.batch_transfer_files(source_endpoint=lab_Greatlakes_gen_scripts.endpoint_id, destination_endpoint=apogee_gen_scripts_folder_bookmark.endpoint_id, filelist_source=endpoint_relative_src_lines, filelist_dest=endpoint_relative_dest_lines)
# _transfer_task
# Synchronously wait for transfer to complete
pending_tasks, (completed_tasks, failed_tasks) = connect_man.batch_transfer_files(source_endpoint=lab_Greatlakes_gen_scripts.endpoint_id, destination_endpoint=apogee_gen_scripts_folder_bookmark.endpoint_id,
																				   filelist_source=endpoint_relative_src_lines, filelist_dest=endpoint_relative_dest_lines,
																				    synchronous_wait=True, label='GL->Apogee Batch Run Log Files')
completed_tasks

Submitted transfer task 2a6e467c-3eca-11ef-9637-453c3ae125a5 with 30 files.
because `synchronous_wait == True`, synchronously waiting for 1 tasks to complete:
Waiting for task 2a6e467c-3eca-11ef-9637-453c3ae125a5 to complete...
Task 2a6e467c-3eca-11ef-9637-453c3ae125a5 completed.
All transfers completed successfully.


{'2a6e467c-3eca-11ef-9637-453c3ae125a5': GlobusHTTPResponse({
   "DATA_TYPE": "transfer_result",
   "code": "Accepted",
   "message": "The transfer has been accepted and a task has been created and queued for execution",
   "request_id": "kPU0lg0SS",
   "resource": "/transfer",
   "submission_id": "2a6e467d-3eca-11ef-9637-453c3ae125a5",
   "task_id": "2a6e467c-3eca-11ef-9637-453c3ae125a5",
   "task_link": {
     "DATA_TYPE": "link",
     "href": "task/2a6e467c-3eca-11ef-9637-453c3ae125a5?format=json",
     "rel": "related",
     "resource": "task",
     "title": "related task"
   }
 })}